# Marginal Likelihood

In [ ]:
%matplotlib inline

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
import exoplanet as xo
import pymc3 as pm
import theano
import theano.tensor as tt
from tqdm import tqdm_notebook as tqdm
from corner import corner

starry.config.lazy = True
starry.config.quiet = True

Load the Earth:

In [ ]:
inc_true = 70.0
map = starry.Map(ydeg=10, inc=inc_true)
map.load("earth")
map.show()

Generate a rotational light curve:

In [ ]:
theta = np.linspace(0, 360, 1000)
flux0 = map.flux(theta=theta).eval()
sigma = 0.0025
flux = flux0 + np.random.randn(len(theta)) * sigma
plt.plot(theta, flux);

Let's solve for the inclination of the body, marginalizing over the map coefficients. We'll assume we know the true variance of the coefficients.

In [ ]:
y_true = map[1:, :].eval()
y_var_true = np.var(y_true)

In [ ]:
# Set the data & prior covariance
map.set_data(flux, C=sigma ** 2)
map.set_prior(L=y_var_true)

# Compile the marginal log likelihood function
inc = tt.dscalar()


def _lnlike(inc):
    map.inc = inc
    return map.lnlike(theta=theta)


lnlike = theano.function([inc], _lnlike(inc))

# Evaluate the likelihood on a grid of inclinations
N = 300
incs = np.linspace(0, 90, N)
ll = np.zeros(N)
for i, inc in tqdm(enumerate(incs), total=N):
    ll[i] = lnlike(inc)

# Plot the result
plt.plot(incs, np.exp(ll - ll.max()))
plt.axvline(inc_true, color="C1")
plt.xlabel("inclination [degrees]", fontsize=14)
plt.ylabel("relative probability", fontsize=14);

The entire thing took 5 seconds, so we were sampling at **60 iterations per second.**

Now let's sample the inclination with `pymc3`.

In [ ]:
with pm.Model() as model:

    map = starry.Map(ydeg=10)
    map.inc = pm.Uniform("inc", lower=0, upper=90)

    map.set_data(flux, C=sigma ** 2)
    map.set_prior(L=y_var_true)

    pm.Potential("marginal", map.lnlike(theta=theta))

In [ ]:
import theano

with model:
    func = xo.utils.get_theano_function_for_var(model.logpt, profile=True)
    grad = xo.utils.get_theano_function_for_var(
        theano.grad(model.logpt, model.vars), profile=True
    )
    args = xo.utils.get_args_for_theano_function()

In [ ]:
grad.profile.summary()

In [ ]:
func.profile.summary()

Optimize to find a starting point:

In [ ]:
%%time
with model:
    map_soln = xo.optimize()

In [ ]:
map_soln["inc"]

Looks about right. Let's sample. We'll do a *very* short run: 250 tuning steps and 500 draws.

In [ ]:
%%time

with model:
    trace = pm.sample(
        tune=250,
        draws=500,
        start=map_soln,
        chains=4,
        cores=1,
        step=xo.get_dense_nuts_step(target_accept=0.9),
    )

That took 30 minutes (!!!) with an average of less than 2 iterations per second. Plus, the posterior disagrees with the grid search.

In [ ]:
display(pm.summary(trace, varnames=["inc"]))

In [ ]:
plt.plot(incs, np.exp(ll - ll.max()), label="grid")
plt.hist(
    trace["inc"],
    histtype="step",
    bins=10,
    weights=(1 / 300) * np.ones(len(trace["inc"])),
    label="pymc3",
)
plt.axvline(inc_true, color="k", ls="--", label="truth")
plt.xlabel("inclination [degrees]", fontsize=14)
plt.ylabel("relative probability", fontsize=14)
plt.legend();